# Assignment 7 : Prediction Using Linear Regression

In [46]:
from google.colab import files as fs

upload = fs.upload()

Saving diamonds.csv to diamonds (1).csv


In [53]:
import pandas as pd

diamond_df = pd.read_csv('diamonds.csv')
diamond_df.head()
diamond_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  53940 non-null  int64  
 1   carat       53940 non-null  float64
 2   cut         53940 non-null  object 
 3   color       53940 non-null  object 
 4   clarity     53940 non-null  object 
 5   depth       53940 non-null  float64
 6   table       53940 non-null  float64
 7   price       53940 non-null  int64  
 8   x           53940 non-null  float64
 9   y           53940 non-null  float64
 10  z           53940 non-null  float64
dtypes: float64(6), int64(2), object(3)
memory usage: 4.5+ MB


# 2. Data Cleaning

In [48]:
missing_values = diamond_df.isnull().sum()
display(missing_values)

,0
Unnamed: 0,0
carat,0
cut,0
color,0
clarity,0
depth,0
table,0
price,0
x,0
y,0


There is no missing data



## Identify outliers



In [49]:
numerical_cols = diamond_df.select_dtypes(include=['float64', 'int64']).columns

outliers_info = {}

for col in numerical_cols:
    Q1 = diamond_df[col].quantile(0.25)
    Q3 = diamond_df[col].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = diamond_df[(diamond_df[col] < lower_bound) | (diamond_df[col] > upper_bound)]
    outliers_count = outliers.shape[0]

    outliers_info[col] = {
        'IQR': IQR,
        'lower_bound': lower_bound,
        'upper_bound': upper_bound,
        'outliers_count': outliers_count
    }

for col, info in outliers_info.items():
    print(f"Column: {col}")
    print(f"  IQR: {info['IQR']:.2f}")
    print(f"  Lower Bound: {info['lower_bound']:.2f}")
    print(f"  Upper Bound: {info['upper_bound']:.2f}")
    print(f"  Outliers Count: {info['outliers_count']}")
    print("-" * 20)

Column: Unnamed: 0
  IQR: 26969.50
  Lower Bound: -26968.50
  Upper Bound: 80909.50
  Outliers Count: 0
--------------------
Column: carat
  IQR: 0.64
  Lower Bound: -0.56
  Upper Bound: 2.00
  Outliers Count: 1889
--------------------
Column: depth
  IQR: 1.50
  Lower Bound: 58.75
  Upper Bound: 64.75
  Outliers Count: 2545
--------------------
Column: table
  IQR: 3.00
  Lower Bound: 51.50
  Upper Bound: 63.50
  Outliers Count: 605
--------------------
Column: price
  IQR: 4374.25
  Lower Bound: -5611.38
  Upper Bound: 11885.62
  Outliers Count: 3540
--------------------
Column: x
  IQR: 1.83
  Lower Bound: 1.96
  Upper Bound: 9.29
  Outliers Count: 32
--------------------
Column: y
  IQR: 1.82
  Lower Bound: 1.99
  Upper Bound: 9.27
  Outliers Count: 29
--------------------
Column: z
  IQR: 1.13
  Lower Bound: 1.22
  Upper Bound: 5.73
  Outliers Count: 49
--------------------


Filter the DataFrame to remove rows where 'x', 'y', or 'z' are zero and then recalculate and display the outlier counts for the numerical columns in the filtered DataFrame.



In [50]:
initial_rows = diamond_df.shape[0]
diamond_data = diamond_df[(diamond_df['x'] != 0) & (diamond_df['y'] != 0) & (diamond_df['z'] != 0)].copy()
rows_after_filtering = diamond_data.shape[0]

print(f"Initial number of rows: {initial_rows}")
print(f"Number of rows after removing zero values in x, y, or z: {rows_after_filtering}")
print(f"Number of rows removed: {initial_rows - rows_after_filtering}")

numerical_cols_filtered = diamond_data.select_dtypes(include=['float64', 'int64']).columns

outliers_info_filtered = {}

for col in numerical_cols_filtered:
    Q1 = diamond_data[col].quantile(0.25)
    Q3 = diamond_data[col].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = diamond_data[(diamond_data[col] < lower_bound) | (diamond_data[col] > upper_bound)]
    outliers_count = outliers.shape[0]

    outliers_info_filtered[col] = {
        'IQR': IQR,
        'lower_bound': lower_bound,
        'upper_bound': upper_bound,
        'outliers_count': outliers_count
    }

print("\nOutliers in filtered data:")
for col, info in outliers_info_filtered.items():
    print(f"Column: {col}")
    print(f"  IQR: {info['IQR']:.2f}")
    print(f"  Lower Bound: {info['lower_bound']:.2f}")
    print(f"  Upper Bound: {info['upper_bound']:.2f}")
    print(f"  Outliers Count: {info['outliers_count']}")
    print("-" * 20)

Initial number of rows: 53940
Number of rows after removing zero values in x, y, or z: 53920
Number of rows removed: 20

Outliers in filtered data:
Column: Unnamed: 0
  IQR: 26969.50
  Lower Bound: -26966.50
  Upper Bound: 80911.50
  Outliers Count: 0
--------------------
Column: carat
  IQR: 0.64
  Lower Bound: -0.56
  Upper Bound: 2.00
  Outliers Count: 1883
--------------------
Column: depth
  IQR: 1.50
  Lower Bound: 58.75
  Upper Bound: 64.75
  Outliers Count: 2543
--------------------
Column: table
  IQR: 3.00
  Lower Bound: 51.50
  Upper Bound: 63.50
  Outliers Count: 604
--------------------
Column: price
  IQR: 4374.25
  Lower Bound: -5612.38
  Upper Bound: 11884.62
  Outliers Count: 3532
--------------------
Column: x
  IQR: 1.83
  Lower Bound: 1.96
  Upper Bound: 9.29
  Outliers Count: 24
--------------------
Column: y
  IQR: 1.82
  Lower Bound: 1.99
  Upper Bound: 9.27
  Outliers Count: 22
--------------------
Column: z
  IQR: 1.13
  Lower Bound: 1.22
  Upper Bound: 5.73
  

## Check for inconsistencies


In [51]:
categorical_cols = diamond_data.select_dtypes(include='object').columns

for col in categorical_cols:
    print(f"Column: {col}")
    display(diamond_data[col].value_counts())
    print("-" * 30)

Column: cut


,count
cut,
Ideal,21548
Premium,13780
Very Good,12081
Good,4902
Fair,1609


------------------------------
Column: color


,count
color,
G,11284
E,9797
F,9538
H,8298
D,6774
I,5421
J,2808


------------------------------
Column: clarity


,count
clarity,
SI1,13063
VS2,12254
SI2,9185
VS1,8170
VVS2,5066
VVS1,3654
IF,1790
I1,738


------------------------------


Ensure all columns have appropriate data types for analysis.


In [52]:
diamond_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 53920 entries, 0 to 53939
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  53920 non-null  int64  
 1   carat       53920 non-null  float64
 2   cut         53920 non-null  object 
 3   color       53920 non-null  object 
 4   clarity     53920 non-null  object 
 5   depth       53920 non-null  float64
 6   table       53920 non-null  float64
 7   price       53920 non-null  int64  
 8   x           53920 non-null  float64
 9   y           53920 non-null  float64
 10  z           53920 non-null  float64
dtypes: float64(6), int64(2), object(3)
memory usage: 4.9+ MB


In [57]:
cat_cols = ['cut', 'color', 'clarity']

df_encoded = pd.get_dummies(diamond_data, columns=cat_cols, drop_first=True)

df_encoded.head()


,Unnamed: 0,carat,depth,table,price,x,y,z,cut_Good,cut_Ideal,...,color_H,color_I,color_J,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,1,0.23,61.5,55.0,326,3.95,3.98,2.43,False,True,...,False,False,False,False,False,True,False,False,False,False
1,2,0.21,59.8,61.0,326,3.89,3.84,2.31,False,False,...,False,False,False,False,True,False,False,False,False,False
2,3,0.23,56.9,65.0,327,4.05,4.07,2.31,True,False,...,False,False,False,False,False,False,True,False,False,False
3,4,0.29,62.4,58.0,334,4.20,4.23,2.63,False,False,...,False,True,False,False,False,False,False,True,False,False
4,5,0.31,63.3,58.0,335,4.34,4.35,2.75,True,False,...,False,False,True,False,False,True,False,False,False,False


In [59]:
diamonds_model = df_encoded.sample(n=12_500)

# 3. Using other columns to determine the price

In [61]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

X = diamonds_model.drop('price', axis=1)
y = diamonds_model['price']

In [65]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [66]:
y_pred = model.predict(X_test)
print("R² Score:", r2_score(y_test, y_pred))

R² Score: 0.9115210808761574


# 4. Select 2 most suitable features using PCA

In [71]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

cont_cols = ['carat', 'depth', 'table', 'x', 'y', 'z']
X = diamonds_model[cont_cols]
y = diamonds_model['price']

In [79]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)


LinearRegression()

In [80]:
y_pred = model.predict(X_test)
print("R² Score:", r2_score(y_test, y_pred))

R² Score: 0.7997916856361742


R² Score: 0.79979**1**...

# 5. Lasso and Ridge Regression

In [75]:
from sklearn.linear_model import Lasso, Ridge
from sklearn.metrics import r2_score

lasso = Lasso(alpha=0.1)
lasso.fit(X_train, y_train)
y_pred_lasso = lasso.predict(X_test)
print("Lasso R² Score:", r2_score(y_test, y_pred_lasso))



Lasso R² Score: 0.7997909696532072


Lasso R² Score: 0.79979**0969**6532072


In [76]:
ridge = Ridge(alpha=1.0)
ridge.fit(X_train, y_train)
y_pred_ridge = ridge.predict(X_test)
print("Ridge R² Score:", r2_score(y_test, y_pred_ridge))


Ridge R² Score: 0.7997905759935886


Ridge R² Score: 0.79979**0575**9935886

# 6. Model comparison

The normal regresion performed well with an R² of 0.911

The others had almost similar results but
PCA performed the best, then came Ridge and Lasso regression came last